In [1]:
## Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

## Train test split
from sklearn.model_selection import train_test_split

## Pipeline
from imblearn.pipeline import Pipeline  #  Pipeline
from sklearn.compose import ColumnTransformer  # Join transformers
from imblearn.over_sampling import SMOTE  # Oversampling technique
from scipy import stats # Exclude outliers

## Model
from sklearn.tree import DecisionTreeClassifier

## Precision metrics
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

## Working with data frames
import pandas as pd
import numpy as np

## Saving the pipeline 
import joblib

In [2]:
# Create data frame of people
pple = (
    pd.read_csv("application_record.csv")  # Read the csv file
    .drop_duplicates('ID', keep='last')  # Drop duplicate ids
    .drop('OCCUPATION_TYPE', axis=1)  # Drop occupation type column
)  # 438 510 rows

In [57]:
# Create data frame of record
record = (
    # Read the csv file
    pd.read_csv("credit_record.csv")  
    # Drop months balance its not needed
    .drop('MONTHS_BALANCE', axis=1)  
     # Replace the text - it means the same thing
    .replace({'C': 0, 'X' : 0}) 
    # Convert to number
    .assign(STATUS=lambda x: pd.to_numeric(x['STATUS']))  
)  # 45985 rows



# Mark the rows where the person owed the debt for two or more monts
# as default marker 1 otherwise 0
record['RESULT'] = record['STATUS'].apply(lambda x:1 if x >= 2 else 0) 

# resulta = df.loc[df['RESULT'] == 1]
# print(resulta.iloc[1])

# Join the two data frames based on the ID inner wise
df = pple.join(record.set_index('ID'), on='ID', how='inner')  # 36457 rows

ID                                           5009628
CODE_GENDER                                        F
FLAG_OWN_CAR                                       N
FLAG_OWN_REALTY                                    N
CNT_CHILDREN                                       0
AMT_INCOME_TOTAL                            238500.0
NAME_INCOME_TYPE                             Working
NAME_EDUCATION_TYPE    Secondary / secondary special
NAME_FAMILY_STATUS                           Married
NAME_HOUSING_TYPE                  House / apartment
DAYS_BIRTH                                    -19305
DAYS_EMPLOYED                                  -3296
FLAG_MOBIL                                         1
FLAG_WORK_PHONE                                    1
FLAG_PHONE                                         0
FLAG_EMAIL                                         0
CNT_FAM_MEMBERS                                  2.0
STATUS                                             5
RESULT                                        

In [4]:
# Features that need to be manipulated with
numeric_features = ['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'CNT_CHILDREN', 
                    'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']

categorical_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 
                        'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 
                        'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE']

In [5]:
for column in numeric_features:
    df[(np.abs(stats.zscore(df[column])) < 3)]

In [6]:
# Apply transformers
numeric_transformer = Pipeline(steps=[
    # Replace missing values with median
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler())])  # Scale normally the values

categorical_transformer = Pipeline(steps=[
    # Replace missing values with the word missing
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    # Encode the categorical values to prevent bias
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])  

In [7]:
# Join the transformers into one function 
transformer = ColumnTransformer(
    transformers = [
        # Mame of the operation, what transformer is applied, 
        # on which features
        ('numeric_data_preprocessing', 
         numeric_transformer, numeric_features),
        ('categorical_data_preprocessing', 
         categorical_transformer, categorical_features),
    ]
)

In [8]:
# Creating a pipeline with specific steps
pipeline = Pipeline(
    steps=[
        # Apply transformation rules on the features
        ('transformer', transformer),  
        # Imbalanced dataset
        ('imbalanced', SMOTE(random_state=38)),  
        # Apply statistical model
        ('rf_estimator', DecisionTreeClassifier())  
    ])

In [9]:
# Preparing before using the pipeline on the dataframe

# Split the data into train and test
# Divide the data frame and the actual results
X = df.drop('RESULT', axis=1) 
y = df['RESULT']

# Split into test, train and data, results
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
# Run the pipeline
pipeline.fit(X_train, y_train)

In [11]:
# Predict 
y_pred = pipeline.predict(X_test)

In [12]:
# Measure accuracy

# # Accuracy Score:  0.9698861408099368
# print("Accuracy Score: ", accuracy_score(y_test, y_pred))
# # F1 Score:  0.9816748804118967
# print("F1 Score: ", f1_score(y_test, y_pred, average='weighted'))
# # Precision Score:  0.9960501501176162
# print("Precision Score: ", precision_score(y_test, y_pred, average='weighted'))
# # Recall Score:  0.9698861408099368
# print("Recall Score: ", recall_score(y_test, y_pred, average='weighted'))

In [15]:
# Sving the pipeline
save_file_name = 'credit_risk_pipeline.pkl'  # Name
save_path = "../backend/core/"  # Path
joblib.dump(pipeline, save_path+save_file_name)  # Persist pipeline

['../backend/core/credit_risk_pipeline.pkl']

In [64]:
# df_predict = pd.DataFrame({
# "ID" :                                             5009628,
# "CODE_GENDER" :                                        "F",
# "FLAG_OWN_CAR" :                                       "N",
# "FLAG_OWN_REALTY" :                                    "N",
# "CNT_CHILDREN" :                                       "0",
# "AMT_INCOME_TOTAL" :                            238500.0,
# "NAME_INCOME_TYPE" :                             "Working",
# "NAME_EDUCATION_TYPE" :    "Secondary / secondary special",
# "NAME_FAMILY_STATUS" :                           "Married",
# "NAME_HOUSING_TYPE" :                  "House / apartment",
# "DAYS_BIRTH" :                                    -19305,
# "DAYS_EMPLOYED" :                                  -3296,
# "FLAG_MOBIL" :                                         1,
# "FLAG_WORK_PHONE" :                                    1,
# "FLAG_PHONE" :                                         0,
# "FLAG_EMAIL" :                                         0,
# "CNT_FAM_MEMBERS" :                                   2.00,
# "STATUS" :                                             5,
# "ANOTHER_FIELD": "jedna"
    
# }, index=[5009628])

# pipeline.predict(df_predict)